In [15]:
import os
# save using vs code
SCALA_VERSION = '2.12'
SPARK_VERSION = '3.1.3'
# Download Kafka Jar file, this for readStream.format("kafka"), "kafka" is a driver
# kafka driver code is part of Maven Jar file
# https://mvnrepository.com/artifact/org.apache.spark/spark-sql-kafka-0-10_2.12/3.1.3
# pyspark-shell shall download the jar file behind..
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-sql-kafka-0-10_{SCALA_VERSION}:{SPARK_VERSION} pyspark-shell'

In [16]:
import findspark
findspark.init()

In [17]:
import pyspark
from pyspark.sql import SparkSession
spark =  SparkSession.builder.master("local[4]").config('spark.sql.shuffle.partitions', 4).appName("usecase1").getOrCreate()

In [18]:
kafkareadDf = spark.readStream.format("kafka").option("kafka.bootstrap.servers", "localhost:9092").option("subscribe", "stock-ticks").option("group.id", "stock-ticks-group3-na123").load()

In [19]:
kafkareadDf.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [20]:
stockDf = kafkareadDf.selectExpr("CAST(value AS STRING)")
stockDf.printSchema()

root
 |-- value: string (nullable = true)



In [21]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructField, StructType, DoubleType, StringType, LongType, TimestampType

schema = StructType([
  StructField("symbol", StringType(), True),
    StructField("volume", LongType(), True),
     StructField("price", DoubleType(), True),
    StructField("timestamp", LongType(),  True)  
    
]
)

In [22]:
jsonStockDf = stockDf.withColumn("value", F.from_json("value",schema))
jsonStockDf.printSchema()

root
 |-- value: struct (nullable = true)
 |    |-- symbol: string (nullable = true)
 |    |-- volume: long (nullable = true)
 |    |-- price: double (nullable = true)
 |    |-- timestamp: long (nullable = true)



In [23]:
valueDf = jsonStockDf.select("value.*")
valueDf.printSchema()


root
 |-- symbol: string (nullable = true)
 |-- volume: long (nullable = true)
 |-- price: double (nullable = true)
 |-- timestamp: long (nullable = true)



In [24]:
#Year=2022/Month=03/Day=18/Hour=01/Symbol=TSLA   for every minute, 1 file
from pyspark.sql.functions import *
timeDf = valueDf.withColumn("SYMBOL1",col("symbol"))\
                .withColumn("timestamp", col("timestamp")/1000)\
                .withColumn("time" , to_timestamp(col("timestamp")))\
                .withColumn("YEAR" ,date_format(col("time"),"yyyy"))\
                .withColumn("MONTH" ,date_format(col("time"),"MM"))\
                .withColumn("DAY" ,date_format(col("time"),"dd"))\
                .withColumn("HOUR" ,date_format(col("time"),"HH"))\
                .withColumn("MIN" ,date_format(col("time"),"mm"))\
                .drop("timestamp")
timeDf.printSchema()




root
 |-- symbol: string (nullable = true)
 |-- volume: long (nullable = true)
 |-- price: double (nullable = true)
 |-- SYMBOL1: string (nullable = true)
 |-- time: timestamp (nullable = true)
 |-- YEAR: string (nullable = true)
 |-- MONTH: string (nullable = true)
 |-- DAY: string (nullable = true)
 |-- HOUR: string (nullable = true)
 |-- MIN: string (nullable = true)



In [25]:

#echoOnconsole = timeDf.writeStream.outputMode("update").option("truncate", False).format("console").start()



In [1]:
timeDf.writeStream.trigger(processingTime='65 seconds').queryName("Write ticks to csv1").format("csv").option("path", "hdfs://localhost:9000/layers/raw/csv/").option("header", True).option("checkpointLocation", "file:///tmp/spark39").partitionBy("year", "month", "day", "hour", "SYMBOL1").option("truncate", False).start()

NameError: name 'timeDf' is not defined

In [ ]:
#def processBatchData(candleBatchDf, batch_id):
#    print ("process batch called", batch_id, "writing ", candleBatchDf.count())
#    (
#      candleBatchDf
#        .select('*')
#        .write
#        .mode("append")
#        .format("csv")
#        .partitionBy("symbol","YEAR", "MONTH", "DAY","HOUR","MIN")
#        .save("hdfs://localhost:9000/Testing4/")
#      )
    
#timeDf.writeStream.outputMode("append")\
#.option("checkpointLocation", "file:///tmp/spark37")\
#.foreachBatch(processBatchData).start()
        
        
  

In [ ]:
#import datetime 
#import time
#order_time = datetime.datetime.now()
#timestamp = int(order_time.timestamp() * 1000)
#print(timestamp)
#print(order_time)